In this example we will create alkyl chains of various lengths, run quantum chemical analysis on each and then replicate them into a simulation cell for an MD simulation

In [2]:
import os 
from pprint import pprint

Check that output from other examples has been generated 

In [9]:
from pathlib2 import Path

In [20]:
need_files = ['methane.xyz']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run structures.ipynb")
        os.system("jupyter nbconvert --to python  structures.ipynb")
        os.system("python structures.py")

In [21]:
need_files = ['ethane.xyz']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run buildingblocks.ipynb")
        os.system("jupyter nbconvert --to python  buildingblocks.ipynb")
        os.system("python buildingblocks.py")

Need to run buildingblocks.ipynb


In [22]:
need_files = ['oplsaa.pkl']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run forcefields.ipynb")
        os.system("jupyter nbconvert --to python  forcefields.ipynb")
        os.system("python forcefields.py")

Need to run forcefields.ipynb


In [23]:
import streamm

Now let's create project and resource to keep track of our work

In [24]:
alkyl_example = streamm.Project('alkyl_example')
res_local = streamm.Resource('local')

Update relative location of templates directory 

In [25]:
res_local.dir['templates'] =  os.path.join(res_local.dir['home'],'..','templates','')

Make sure this is the location of the templates directory that comes with the streamm git repository  https://github.com/NREL/streamm-tools

In [26]:
print res_local.dir['templates']

/Users/tkemper/Development/streamm-tools/examples/../templates/


Create the local directories that will store our files 

In [27]:
res_local.make_dir()

Tell the project about our directories 

In [28]:
alkyl_example.set_resource(res_local)

Read in the methane.xyz file created in the structure.ipynb example 

In [29]:
methane = streamm.Buildingblock('methane')

In [30]:
methane.read_xyz()

Create the neighbor list 

In [31]:
methane.bonded_nblist = methane.guess_nblist(0,radii_buffer=1.25)

and the bonded interactions

In [32]:
methane.bonded_bonds()
methane.bonded_angles()
methane.bonded_dih()

In [33]:
print methane.n_particles

5


In [34]:
print methane.print_properties()

 n_particles:5 
 n_bonds:4
 n_angles:6
 n_dihedrals:0
 n_impropers:0


Set the paramkeys so we can identify force field parameters later on

In [35]:
for pkey,p in methane.particles.iteritems():
    if( p.symbol == 'C' ):
        p.paramkey = 'CT'
    elif( p.symbol == 'H' ):
        p.paramkey = 'HC'

In [36]:
for pk,p in methane.particles.iteritems():
    p.residue = 1
    p.resname = 'METH'

Set some rsites to be able to join molecules together 

In [37]:
methane.particles[1].rsite = 'RH'
methane.particles[2].rsite = 'RH'

In [38]:
methane.find_rsites()

In [39]:
print methane.show_rsites()

rsite:RH[ paticle:atom[1] H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom[2] H (H) index:2 n_bonds:1] 



Read in ethane.xyz from the buildinblock.ipynb example 

In [40]:
ethane = streamm.Buildingblock('ethane')

In [41]:
ethane.read_xyz()

Guess bonded neighbor list based on `bonded_radii`

In [42]:
ethane.bonded_nblist = ethane.guess_nblist(0,radii_buffer=1.25)

In [43]:
ethane.bonded_bonds()
ethane.bonded_angles()
ethane.bonded_dih()

In [44]:
print ethane.print_properties()

 n_particles:8 
 n_bonds:7
 n_angles:12
 n_dihedrals:9
 n_impropers:0


Set the `paramkey`'s  as described in the force field example

In [45]:
for pkey,p in ethane.particles.iteritems():
    if( p.symbol == 'C' ):
        p.paramkey = 'CT'
    elif( p.symbol == 'H' ):
        p.paramkey = 'HC'

Set the `resname` of each particle to `ETH` 

In [46]:
for pk,p in ethane.particles.iteritems():
    p.residue = 1
    p.resname = 'ETH'

Set `rsite`'s to hydrogens to be replaced during join

In [47]:
ethane.particles[1].rsite = 'RH'
ethane.particles[5].rsite = 'RH'

Run `find_rsites()` to populate `func` list

In [48]:
ethane.find_rsites()

In [49]:
print ethane.show_rsites()

rsite:RH[ paticle:atom[1] H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom[5] H (H) index:5 n_bonds:1] 



In [50]:
import copy

Create octane from ethane 

Copy ethane to a new Buildingblock octane 

In [51]:
octane = copy.deepcopy(ethane)

In [52]:
from streamm.structures.buildingblock import attach

Then attach 3 more ethanes to make an octane 

In [53]:
for i in range(3):
    octane = attach(octane,ethane,'RH',1,'RH',0)

Update the tag

In [54]:
octane.tag = 'octane'

Rename the residue and resname for octane

In [55]:
for pk,p in octane.particles.iteritems():
    p.residue = 2
    p.resname = "OCT"
 

In [56]:
octane.write_xyz()

Print new `rsite`'s 

In [57]:
print octane.show_rsites()

rsite:RH[ paticle:atom[1] H (H) index:1 n_bonds:1] 
rsite:RH[ paticle:atom[23] H (H) index:23 n_bonds:1] 



Find the 4th carbon to attach an ethane 

In [58]:
print octane.particles[14].symbol

H


In [59]:
octane.particles[14].rsite = 'R2'

In [60]:
octane.find_rsites()

Attach the ethane to the fourth carbon to make 4-ethyloctane

In [61]:
ethyl_octane = attach(octane,ethane,'R2',0,'RH',0)

In [62]:
ethyl_octane.tag = '4-ethyloctane'

In [63]:
ethyl_octane.write_xyz()

Read in pickled oplsaa parameters from forcefield example 

In [64]:
oplsaa = streamm.forcefields.parameters.read_pickle('oplsaa')

In [65]:
print oplsaa


    Parameters 
      LJ parameters 2 
      Bond parameters 2 
      Angle parameters 2 
      Dihedral parameters 1 
      Improper Dihedral parameters 0 



Create NWChem Calculation object 

In [66]:
nwchem_i = streamm.NWChem('nw_ethane_HF')

Add calculation to project

In [67]:
alkyl_example.add_calc(nwchem_i)

Set the structure of the calculation to ethane

In [68]:
nwchem_i.strucC = ethane

Set the resource to be local

In [69]:
nwchem_i.set_resource(res_local)

Make the local directories 

In [70]:
nwchem_i.make_dir()

Change to the `scratch` directory

In [71]:
os.chdir(nwchem_i.dir['scratch'])

Copy the template files to the scratch direcotry 

In [72]:
file_type = 'templates'
file_key = 'run'
file_name = "nwchem.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
nwchem_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [73]:
file_type = 'templates'
file_key = 'nw'
file_name = "nwchem.nw"
from_dirkey = 'templates'
to_dirkey = 'scratch'
nwchem_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

Read in the template files and add them to the `str` dictionary

In [74]:
nwchem_i.load_str('templates','nw')        
nwchem_i.load_str('templates','run')

Set the properties dictionary to desired calculation details

In [75]:
nwchem_i.properties['basis'] = '6-31g'
nwchem_i.properties['method'] = 'UHF'
nwchem_i.properties['charge'] = 0
nwchem_i.properties['spin_mult'] = 1
nwchem_i.properties['task'] = 'SCF '
nwchem_i.properties['coord'] = nwchem_i.strucC.write_coord()

In [76]:
pprint(nwchem_i.properties)

{u'allocation': u'',
 u'basis': u'6-31g',
 u'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 u'coord': u'     C       1.34000000      -0.00000000       0.00000000 \n     H       1.74000000      -0.00000000      -1.13137084 \n     H       1.74000000       0.97979589       0.56568542 \n     H       1.74000000      -0.97979589       0.56568542 \n     C       0.00000000       0.00000000       0.00000000 \n     H      -0.40000000       0.00000000       1.13137084 \n     H      -0.40000000      -0.97979589      -0.56568542 \n     H      -0.40000000       0.97979589      -0.56568542 \n',
 u'exe_command': u'./',
 u'feature': u'24core',
 u'finish_str': u'Total times  cpu:',
 u'method': u'UHF',
 u'nodes': 1,
 u'nproc': 1,
 u'pmem': 1500,
 u'ppn': 1,
 u'queue': u'batch',
 u'scratch': u'/Users/tkemper/Development/streamm-tools/examples/scratch/nw_ethane_HF/',
 u'spin_mult': 1,
 u'task': u'SCF ',
 'uncompress': 'tar -xzf ',
 u'walltime': 24}


Replace the keys in the template strings and write the input files 

In [77]:
nwchem_i.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_i.tag))

Add the input file to the properties to be written into the run file

In [78]:
nwchem_i.properties['input_nw'] = nwchem_i.files['input']['nw']
nwchem_i.replacewrite_prop('run','scripts','run','%s.sh'%(nwchem_i.tag))

Add the log file to the files dictionary 

In [79]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(nwchem_i.tag)
nwchem_i.add_file(file_type,file_key,file_name)

Change back to the root directory and write a json file

In [80]:
os.chdir(nwchem_i.dir['home'])
alkyl_example.dump_json()

Change back to scratch

In [81]:
os.chdir(nwchem_i.dir['scratch'])

Run the bash script for the calculation or submit the job to the cluster

In [82]:
nwchem_i.run()

Check the status of all the calculations in the project

In [83]:
alkyl_example.check()

Calculation nw_ethane_HF has status running


Run the analysis

In [84]:
nwchem_i.analysis()

Tar and zip the results and copy them to a storage location

In [85]:
nwchem_i.store()

Save json in home directory

In [86]:
os.chdir(nwchem_i.dir['home'])
alkyl_example.dump_json()

Create a Gaussian Calculation object 

In [87]:
gaussian_i = streamm.Gaussian('gaus_ethane_HF')

Add the calculation to the project

In [88]:
alkyl_example.add_calc(gaussian_i)

Set the structure of the calculation to ethane

In [89]:
gaussian_i.strucC = ethane

Set the resource to be local

In [90]:
gaussian_i.set_resource(res_local)

Make the local directories 

In [91]:
gaussian_i.make_dir()

Copy the template files to the scratch direcotry 

In [92]:
os.chdir(gaussian_i.dir['scratch'])

Copy the template files to the scratch direcotry 

In [93]:
file_type = 'templates'
file_key = 'run'
file_name = "gaussian.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
gaussian_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [94]:
file_type = 'templates'
file_key = 'com'
file_name = "gaussian.com"
from_dirkey = 'templates'
to_dirkey = 'scratch'
gaussian_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

Read in the template files and add them to the `str` dictionary

In [95]:
gaussian_i.load_str('templates','com')        
gaussian_i.load_str('templates','run')

Set the properties dictionary to desired calculation details

In [96]:
gaussian_i.properties['commands'] = 'HF/3-21G SP'
gaussian_i.properties['method'] = 'UHF'
gaussian_i.properties['charge'] = 0
gaussian_i.properties['spin_mult'] = 1
gaussian_i.properties['coord'] = gaussian_i.strucC.write_coord()

In [97]:
pprint(gaussian_i.properties)

{u'allocation': u'',
 u'charge': 0,
 u'commands': u'HF/3-21G SP',
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 u'coord': u'     C       1.34000000      -0.00000000       0.00000000 \n     H       1.74000000      -0.00000000      -1.13137084 \n     H       1.74000000       0.97979589       0.56568542 \n     H       1.74000000      -0.97979589       0.56568542 \n     C       0.00000000       0.00000000       0.00000000 \n     H      -0.40000000       0.00000000       1.13137084 \n     H      -0.40000000      -0.97979589      -0.56568542 \n     H      -0.40000000       0.97979589      -0.56568542 \n',
 u'exe_command': u'./',
 u'feature': u'24core',
 u'finish_str': u'Normal termination of Gaussian',
 u'method': u'UHF',
 u'nodes': 1,
 u'nproc': 1,
 u'pmem': 1500,
 u'ppn': 1,
 u'queue': u'batch',
 u'scratch': u'/Users/tkemper/Development/streamm-tools/examples/scratch/gaus_ethane_HF/',
 u'spin_mult': 1,
 'uncompress': 'tar -xzf ',
 u'walltime': 24}


Replace the keys in the template strings and write the input files 

In [98]:
gaussian_i.replacewrite_prop('com','input','com','%s.com'%(gaussian_i.tag))

Add the input file to the properties to be written into the run file

In [99]:
gaussian_i.properties['input_com'] = gaussian_i.files['input']['com']
gaussian_i.replacewrite_prop('run','scripts','run','%s.sh'%(gaussian_i.tag))

Add the log file to the files dictionary 

In [100]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(gaussian_i.tag)
gaussian_i.add_file(file_type,file_key,file_name)

Change back to the root directory and write a json file

In [101]:
os.chdir(gaussian_i.dir['home'])
alkyl_example.dump_json()

Change back to scratch

In [102]:
os.chdir(gaussian_i.dir['scratch'])

Run the bash script for the calculation or submit the job to the cluster

In [103]:
gaussian_i.run()

Check the status of all the calculations in the project

In [104]:
alkyl_example.check()

Calculation nw_ethane_HF has status running
Calculation gaus_ethane_HF has status running


Run the analysis

In [105]:
os.chdir(alkyl_example.dir['home'])
alkyl_example.dump_json()

Create a LAMMPS Calculation object 

In [106]:
lmp_alkyl = streamm.LAMMPS('lmp_alkyl')

Turn periodic boundries on in all three directions

In [114]:
lmp_alkyl.strucC.lat.pbcs = [True,True,True]

Run the `add_struc()` function to create 10 randomly placed 4-ethyloctane molecules

In [115]:
seed = 92734
lmp_alkyl.strucC = streamm.add_struc(lmp_alkyl.strucC,ethyl_octane,10,seed)

No overlap found adding structure 0
No overlap found adding structure 1
No overlap found adding structure 2
No overlap found adding structure 3
No overlap found adding structure 4
No overlap found adding structure 5
No overlap found adding structure 6
No overlap found adding structure 7
No overlap found adding structure 8
No overlap found adding structure 9
Max placments 10 exceeded resetting to original system 
No overlap found adding structure 0
No overlap found adding structure 1
No overlap found adding structure 2
No overlap found adding structure 3
No overlap found adding structure 4
No overlap found adding structure 5
No overlap found adding structure 6
No overlap found adding structure 7
No overlap found adding structure 8
Max placments 10 exceeded resetting to original system 
No overlap found adding structure 0
No overlap found adding structure 1
No overlap found adding structure 2
No overlap found adding structure 3
No overlap found adding structure 4
No overlap found adding 

The `add_struc()` function randomly places each molecule in a space defined by the lattice of the lmp_alkyl.strucC, then randomly rotates it. 

Then the function checks to make sure it does not overlap any other particles that are already in the lmp_alkyl.strucC. 

If an overlap is found a new position and rotation is chosen until the max placements are exceeded, then the entire system is cleared, and the placement starts again. If the maximum restarts are exceeded, then the size of the lattice is increased, until all the molecules have been added.

Check the lattice see if it expanded 

In [116]:
print lmp_alkyl.strucC.lat

50.000000 0.000000 0.000000
0.000000 50.000000 0.000000
0.000000 0.000000 50.000000


Find the maximum molecule index 

In [117]:
print lmp_alkyl.strucC.n_molecules()

9


In [118]:
print ethyl_octane.tag

4-ethyloctane


Update the structure tag

In [119]:
lmp_alkyl.strucC.tag = ethyl_octane.tag + '_x10'

Write the structure to an xyz file

In [120]:
lmp_alkyl.strucC.write_xyz()

Add 10 ethane to the structure container

In [121]:
seed = 283674
lmp_alkyl.strucC = streamm.add_struc(lmp_alkyl.strucC,ethane,10,seed)

No overlap found adding structure 0
Max placments 10 exceeded resetting to original system 
No overlap found adding structure 0
Max placments 10 exceeded resetting to original system 
No overlap found adding structure 0
Max placments 10 exceeded resetting to original system 
No overlap found adding structure 0
No overlap found adding structure 1
No overlap found adding structure 2
No overlap found adding structure 3
No overlap found adding structure 4
No overlap found adding structure 5
No overlap found adding structure 6
No overlap found adding structure 7
No overlap found adding structure 8
No overlap found adding structure 9


In [122]:
print lmp_alkyl.strucC.n_molecules()

19


Update tag

In [123]:
lmp_alkyl.strucC.tag += '_ethane_x10'

Add 50 methane to structure container using the `add_struc_grid()` which places solvent on grid 

In [124]:
lmp_alkyl.strucC = streamm.add_struc_grid(lmp_alkyl.strucC,methane,50)

Check to see if the lattice was expanded 

In [125]:
print lmp_alkyl.strucC.lat

73.205000 0.000000 0.000000
0.000000 73.205000 0.000000
0.000000 0.000000 73.205000


Update tag

In [126]:
lmp_alkyl.strucC.tag += '_methane_x50'

In [127]:
lmp_alkyl.strucC.write_xyz()

Print all the particles in the structure container 

In [128]:
for pk,p in lmp_alkyl.strucC.particles.iteritems():
    print p,p.paramkey,p.mol,p.residue,p.resname

atom[0] C (C) CT 0 2 OCT
atom[1] H (H) HC 0 2 OCT
atom[2] H (H) HC 0 2 OCT
atom[3] H (H) HC 0 2 OCT
atom[4] C (C) CT 0 2 OCT
atom[5] H (H) HC 0 2 OCT
atom[6] H (H) HC 0 2 OCT
atom[7] C (C) CT 0 2 OCT
atom[8] H (H) HC 0 2 OCT
atom[9] H (H) HC 0 2 OCT
atom[10] C (C) CT 0 2 OCT
atom[11] H (H) HC 0 2 OCT
atom[12] H (H) HC 0 2 OCT
atom[13] C (C) CT 0 2 OCT
atom[14] H (H) HC 0 2 OCT
atom[15] C (C) CT 0 2 OCT
atom[16] H (H) HC 0 2 OCT
atom[17] H (H) HC 0 2 OCT
atom[18] C (C) CT 0 2 OCT
atom[19] H (H) HC 0 2 OCT
atom[20] H (H) HC 0 2 OCT
atom[21] C (C) CT 0 2 OCT
atom[22] H (H) HC 0 2 OCT
atom[23] H (H) HC 0 2 OCT
atom[24] H (H) HC 0 2 OCT
atom[25] C (C) CT 0 1 ETH
atom[26] H (H) HC 0 1 ETH
atom[27] H (H) HC 0 1 ETH
atom[28] C (C) CT 0 1 ETH
atom[29] H (H) HC 0 1 ETH
atom[30] H (H) HC 0 1 ETH
atom[31] H (H) HC 0 1 ETH
atom[32] C (C) CT 1 2 OCT
atom[33] H (H) HC 1 2 OCT
atom[34] H (H) HC 1 2 OCT
atom[35] H (H) HC 1 2 OCT
atom[36] C (C) CT 1 2 OCT
atom[37] H (H) HC 1 2 OCT
atom[38] H (H) HC 1 2 

Set ff parameters for all the bonds, bond angles and dihedrals in the structure container

In [129]:
lmp_alkyl.set_ffparam()

Add template files to calculations

In [130]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_spneut.in"
from_dirkey = 'templates'
to_dirkey = 'scratch'
lmp_alkyl.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [140]:
pprint("Calculation:{} has status:{}".format(lmp_alkyl.tag,lmp_alkyl.meta['status']))

u'Calculation:lmp_alkyl has status:written'


Calculate the center mass of structure

In [141]:
lmp_alkyl.strucC.calc_center_mass()

Create groups out of the molecules

In [142]:
groupset_i = streamm.Groups('mol',lmp_alkyl.strucC)
groupset_i.group_prop('mol','group_mol')

Caculate the ceneter of mass, radius and asphericity of each group

In [143]:
groupset_i.calc_cent_mass()
groupset_i.calc_radius_asphericity()
groupset_i.calc_dl()

Write the center of mass of each group to an .xyz file for visulization 

In [144]:
groupset_i.write_cm_xyz()

In [145]:
import numpy as np

In [146]:
print np.mean(groupset_i.radius),groupset_i.strucC.unit_conf['length']

1.79932546227 ang


In [147]:
print groupset_i.strucC.lat.pbcs

[True, True, True]


Create a neighbor list of groups 

In [148]:
groupset_i.group_nblist.radii_nblist(groupset_i.strucC.lat,groupset_i.cent_mass,groupset_i.radius,radii_buffer=5.25)

Apply periodic boundries to all the groups, so the molecules are not split across pbc's

In [149]:
groupset_i.group_pbcs()

Loop over each group, shift the group to the center of the simulation cell and write an .xyz file that includes the neighbors of the group.

In [150]:
for gk_i,g_i in groupset_i.groups.iteritems():
    if( len(g_i.pkeys) == 32 ):
        print g_i.tag,groupset_i.group_nblist.calc_nnab(gk_i),g_i.mol 
        print g_i.cent_mass
        list_i = []
        for g_j in groupset_i.group_nblist.getnbs(gk_i):
            list_i += groupset_i.groups[g_j].pkeys
        groupset_i.strucC.shift_pos(-1.0*g_i.cent_mass)  # Place center of mass at origin
        groupset_i.strucC.write_xyz_list(list_i,xyz_file='{}_blob.xyz'.format(g_i.tag))
        groupset_i.strucC.shift_pos(g_i.cent_mass)  # Return center of mass 
        

group_mol_0 35 0
[ 11.452512   7.190697   5.926503]
group_mol_1 31 1
[ 14.20855   27.216498  46.743642]
group_mol_2 39 2
[ 25.506379   2.145656  40.697004]
group_mol_3 31 3
[ 48.990649  11.354279  42.633871]
group_mol_4 28 4
[ 39.132369   0.564871  14.682747]
group_mol_5 27 5
[ 33.681792  21.768119  26.826298]
group_mol_6 28 6
[  1.91345   35.78647   40.494419]
group_mol_7 26 7
[ 12.996395  30.128546  26.504759]
group_mol_8 28 8
[  2.914782  18.064497  15.529658]
group_mol_9 34 9
[ 34.541826  34.517255  15.226652]


Fancy aye! 